In [1]:
from spheres import *
from spheres.circuits import * 

from pytket.qiskit import tk_to_qiskit
from pytket.backends.ibm import AerBackend, AerStateBackend, IBMQBackend

from qiskit import QuantumCircuit, QuantumRegister, Aer, execute, IBMQ, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel, QuantumError, ReadoutError, pauli_error, depolarizing_error, thermal_relaxation_error

n_qubits = 2
depth = 20
n_shots = 8000 

n_copies = 3
every = 2
pairwise = True
reuse_cntrls = False

latex = False
use_remote_simulator = True
error_mitigation = False

if error_mitigation or use_remote_simulator:
    provider = IBMQ.load_account()
    job_manager = IBMQJobManager()

In [2]:
circ_specification = random_circuit(n_qubits=n_qubits, depth=depth)
circ = build_circuit(circ_specification)
if latex:
    circ.to_latex_file("circ.tex")

In [3]:
sym_circ_info = symmetrize_circuit(circ_specification,
                                   n_copies=n_copies,
                                   every=every, 
                                   pairwise=pairwise,
                                   reuse_cntrls=reuse_cntrls)
sym_circ = sym_circ_info["circuit"]
if latex:
    sym_circ.to_latex_file("sym_circ.tex")

total_sym_qubits = len(sym_circ.qubits)
print("%d total qubits for symmetrized circuit" % total_sym_qubits)

17 total qubits for symmetrized circuit


In [4]:
analytic_circ = circ.copy()
state_backend = AerStateBackend()
AerStateBackend().compile_circuit(analytic_circ)
circ_distribution = state_backend.get_result(state_backend.process_circuit(analytic_circ)).get_distribution()

print("analytical dist for original circuit:")
for bitstr, prob in circ_distribution.items():
    print("  %s: %f" % ("".join([str(b) for b in bitstr]), prob))

analytical dist for original circuit:
  00: 0.130362
  01: 0.255362
  10: 0.244638
  11: 0.369638


In [5]:
def build_noise_model(n_qubits=None, on_qubits=None):
    # T1 and T2 values for qubits
    T1s = np.random.normal(50e3, 10e3, len(on_qubits)) # Sampled from normal distribution mean 50 microsec
    T2s = np.random.normal(70e3, 10e3, len(on_qubits)) # Sampled from normal distribution mean 50 microsec

    # Truncate random T2s <= T1s
    T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(len(on_qubits))])

    # Instruction times (in nanoseconds)
    time_u1 = 0   # virtual gate
    time_u2 = 50  # (single X90 pulse)
    time_u3 = 100 # (two X90 pulses)
    time_cx = 300
    time_reset = 1000  # 1 microsecond
    time_measure = 1000 # 1 microsecond

    # QuantumError objects
    errors_reset = [thermal_relaxation_error(t1, t2, time_reset)
                    for t1, t2 in zip(T1s, T2s)]
    errors_measure = [thermal_relaxation_error(t1, t2, time_measure)
                    for t1, t2 in zip(T1s, T2s)]
    errors_u1  = [thermal_relaxation_error(t1, t2, time_u1)
                for t1, t2 in zip(T1s, T2s)]
    errors_u2  = [thermal_relaxation_error(t1, t2, time_u2)
                for t1, t2 in zip(T1s, T2s)]
    errors_u3  = [thermal_relaxation_error(t1, t2, time_u3)
                for t1, t2 in zip(T1s, T2s)]
    errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
                thermal_relaxation_error(t1b, t2b, time_cx))
                for t1a, t2a in zip(T1s, T2s)]
                for t1b, t2b in zip(T1s, T2s)]

    # Add errors to noise model
    noise_thermal = NoiseModel()
    for qubit in list(range(n_qubits)):
        if qubit not in on_qubits:
            noise_thermal.add_quantum_error(depolarizing_error(0.000001, 1), ['u1'], [qubit])
            
    for j, q1 in enumerate(on_qubits):
        noise_thermal.add_quantum_error(errors_reset[j], "reset", [q1])
        noise_thermal.add_quantum_error(errors_measure[j], "measure", [q1])
        noise_thermal.add_quantum_error(errors_u1[j], "u1", [q1])
        noise_thermal.add_quantum_error(errors_u2[j], "u2", [q1])
        noise_thermal.add_quantum_error(errors_u3[j], "u3", [q1])
        for k, q2 in enumerate(on_qubits):
            noise_thermal.add_quantum_error(errors_cx[j][k], "cx", [q1, q2])
    return noise_thermal

circ_noise_model = build_noise_model(n_qubits=n_qubits, on_qubits=list(range(n_qubits)))
sym_circ_noise_model = build_noise_model(n_qubits=total_sym_qubits, on_qubits=list(range(len(sym_circ_info["cntrl_qubits"]), total_sym_qubits)))
#sym_circ_noise_model = build_noise_model(n_qubits=total_sym_qubits, on_qubits=list(range(total_sym_qubits)))

In [6]:
def error_calibration(n_qubits, noise_model, use_remote_simulator=False):
    meas_calibs, state_labels = complete_meas_cal(qr=QuantumRegister(n_qubits), circlabel='mcal')
    if not use_remote_simulator:
        noise_job = execute(meas_calibs, backend=Aer.get_backend("qasm_simulator"), shots=n_shots, noise_model=noise_model)
        cal_results = noise_job.result()
    else:
        noise_job = job_manager.run(meas_calibs, backend=provider.get_backend("ibmq_qasm_simulator"), shots=n_shots, noise_model=noise_model)
        cal_results = noise_job.results().combine_results()                                  
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
    return meas_fitter.filter

In [7]:
def qiskit_pytket_counts(counts):
    return dict([(tuple([int(b) for b in "".join(bitstr.split())][::-1]), count) for bitstr, count in counts.items()])

def eval_circuit(circuit, noise_model=None, shots=8000, backend=None):
    circ = circuit.copy()
    AerBackend().compile_circuit(circ)
    qis_circ = tk_to_qiskit(circ)
    counts = execute(qis_circ,\
                     backend=Aer.get_backend('qasm_simulator') if not backend else backend,\
                     noise_model=noise_model,\
                     shots=shots).result().get_counts()
    return qiskit_pytket_counts(counts)

In [8]:
noisy_circ_counts = eval_circuit(circ.copy().measure_all(), noise_model=circ_noise_model, shots=n_shots)
noisy_circ_dists = probs_from_counts(noisy_circ_counts)

print("noisy dist for original circuit:")
for bitstr, prob in noisy_circ_dists.items():
    print("  %s: %f" % ("".join([str(b) for b in bitstr]), prob))

noisy dist for original circuit:
  00: 0.163500
  10: 0.239875
  01: 0.249500
  11: 0.347125


In [9]:
if error_mitigation:
    circ_meas_filter = error_calibration(n_qubits, circ_noise_model)
    noisy_circ_counts = dict([("".join([str(b) for b in bitstr]), count) for bitstr, count in noisy_circ_counts.items()])
    mitigated_circ_counts = circ_meas_filter.apply(noisy_circ_counts)
    mitigated_circ_dists = probs_from_counts(mitigated_circ_counts)

    print("mitigated dist for original circuit:")
    for bitstr, prob in mitigated_circ_dists.items():
        print("  %s: %f" % (bitstr, prob))

In [10]:
def display_sym_dists(sym_dists):
    exp_dists, avg_dist = sym_dists["exp_dists"], sym_dists["avg_dist"], 
    print("  postselected dists:")
    for i, dist in enumerate(exp_dists):
        print("    experiment %d:" % i)
        for bitstr, prob in dist.items():
            print("      %s: %f" % ("".join([str(b) for b in bitstr]), prob))
    print("  averaged dists:")
    for bitstr, prob in avg_dist.items():
        print("    %s: %f" % ("".join([str(b) for b in bitstr]), prob)) 

In [11]:
clean_sym_circ_counts = eval_circuit(sym_circ,\
                                     shots=n_shots,\
                                     backend=provider.get_backend("ibmq_qasm_simulator")\
                                                if use_remote_simulator else None)
print("clean sym circ dists:")
display_sym_dists(process_sym_counts(sym_circ_info, clean_sym_circ_counts))

clean sym circ dists:
  postselected dists:
    experiment 0:
      00: 0.134000
      10: 0.244125
      01: 0.256125
      11: 0.365750
    experiment 1:
      00: 0.126625
      10: 0.238375
      01: 0.253750
      11: 0.381250
    experiment 2:
      00: 0.127125
      10: 0.236625
      01: 0.266000
      11: 0.370250
  averaged dists:
    00: 0.129250
    10: 0.239708
    01: 0.258625
    11: 0.372417


In [12]:
noisy_sym_circ_counts = eval_circuit(sym_circ,\
                                        noise_model=sym_circ_noise_model, 
                                        shots=n_shots,\
                                        backend=provider.get_backend("ibmq_qasm_simulator")\
                                                if use_remote_simulator else None)

print("noisy sym circ dists:")
noisy_sym_circ_dists = process_sym_counts(sym_circ_info, noisy_sym_circ_counts)
display_sym_dists(noisy_sym_circ_dists)

noisy sym circ dists:
  postselected dists:
    experiment 0:
      00: 0.171119
      10: 0.232381
      01: 0.245279
      11: 0.351221
    experiment 1:
      00: 0.172271
      10: 0.238830
      01: 0.256794
      11: 0.332105
    experiment 2:
      00: 0.181944
      10: 0.241594
      01: 0.243436
      11: 0.333026
  averaged dists:
    00: 0.175111
    10: 0.237602
    01: 0.248503
    11: 0.338784


In [13]:
if error_mitigation:
    sym_circ_meas_filter = error_calibration(total_sym_qubits, sym_circ_noise_model, use_remote_simulator=use_remote_simulator)
    noisy_sym_circ_counts = dict([("".join([str(b) for b in bitstr]), count) for bitstr, count in noisy_sym_circ_counts.items()])
    mitigated_sym_circ_counts = sym_circ_meas_filter.apply(noisy_sym_circ_counts)
    mitigated_sym_circ_counts = dict([(tuple([int(b) for b in bitstr]), count) for bitstr, count in mitigated_sym_circ_counts.items()])
    mitigated_sym_circ_dists = process_sym_counts(sym_circ_info, mitigated_sym_circ_counts)

    print("mitigated sym circ dists:")
    display_sym_dists(mitigated_sym_circ_dists)

In [14]:
basis = list(product([0,1], repeat=n_qubits))
expected_probs = []
actual_circ_probs = []
actual_sym_circ_probs = []
for b in basis:
    if b in circ_distribution:
        expected_probs.append(circ_distribution[b])
    else:
        expected_probs.append(0)
    bs = "".join([str(b_) for b_ in b]) 
    if error_mitigation:
        if bs in mitigated_circ_dists.keys():
            actual_circ_probs.append(mitigated_circ_dists[bs])
        else:
            actual_circ_probs.append(0)    
        if b in mitigated_sym_circ_dists["avg_dist"]:
            actual_sym_circ_probs.append(mitigated_sym_circ_dists["avg_dist"][b])
        else:
            actual_sym_circ_probs.append(0)
    else:
        if b in noisy_circ_dists.keys():
            actual_circ_probs.append(noisy_circ_dists[b])
        else:
            actual_circ_probs.append(0)    
        if b in noisy_sym_circ_dists["avg_dist"]:
            actual_sym_circ_probs.append(noisy_sym_circ_dists["avg_dist"][b])
        else:
            actual_sym_circ_probs.append(0)

print("expected results: %s" % expected_probs)
print("actual circ results: %s" % actual_circ_probs)
print("actual sym_circ results: %s" % actual_sym_circ_probs)
print()
print("circ error: %f" % np.linalg.norm(np.array(actual_circ_probs) - np.array(expected_probs)))
print("sym circ error: %f" % np.linalg.norm(np.array(actual_sym_circ_probs) - np.array(expected_probs)))

expected results: [0.13036165235168148, 0.25536165235168157, 0.24463834764831832, 0.36963834764831865]
actual circ results: [0.1635, 0.2495, 0.239875, 0.347125]
actual sym_circ results: [0.1751113158298787, 0.24850299401197604, 0.23760171963764778, 0.3387839705204974]

circ error: 0.040768
sym circ error: 0.055237
